In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchinfo import summary
import numpy as np
import pandas as pd

from datetime import datetime as dt
import os
import time
import sys
sys.path.append('..')

from utils.functions import *
from models.TryNets import *

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)
device = 'cpu'

In [2]:
train_data   = CustomMawiDataset(year='2019', month='01', as_matrix=False)
test_data    = CustomMawiDataset(year='2019', month='02', as_matrix=False)

torch.Size([938330, 48])
torch.Size([938330])
torch.Size([905578, 48])
torch.Size([905578])


In [31]:
class MawiNet(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        
        self.expand_to = 64

        self.adaptation = nn.Sequential(
            nn.Linear(48, self.expand_to ** 2),
            nn.ReLU(),)
        
        self.backbone = nn.Sequential(
            nn.Sequential(
                nn.Conv2d(1, 96, kernel_size=3, stride=1, padding=0),
                nn.BatchNorm2d(96),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3, stride=1)),
                       
            nn.Sequential(
                nn.Conv2d(96, 256, kernel_size=3, stride=1, padding=0),
                nn.BatchNorm2d(256),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3, stride=1)),   
            
            nn.Sequential(
                nn.Conv2d(256, 96, kernel_size=7, stride=2, padding=0),
                nn.BatchNorm2d(96),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=7, stride=2)),            
        )
        
        self.exit = nn.Sequential(
            nn.Flatten(),
            nn.Linear(9600, num_classes),
        )
    
    def unflat(self, x):
        return x.view(-1, 1, self.expand_to, self.expand_to)
    
    def forward(self, x):
        x = self.unflat(self.adaptation(x))
        return self.exit(self.backbone(x))
    
model = MawiNet().to(device)    
# summary(model, (1, 48))
summary(model)

Layer (type:depth-idx)                   Param #
MawiNet                                  --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       200,704
│    └─ReLU: 2-2                         --
├─Sequential: 1-2                        --
│    └─Sequential: 2-3                   --
│    │    └─Conv2d: 3-1                  960
│    │    └─BatchNorm2d: 3-2             192
│    │    └─ReLU: 3-3                    --
│    │    └─MaxPool2d: 3-4               --
│    └─Sequential: 2-4                   --
│    │    └─Conv2d: 3-5                  221,440
│    │    └─BatchNorm2d: 3-6             512
│    │    └─ReLU: 3-7                    --
│    │    └─MaxPool2d: 3-8               --
│    └─Sequential: 2-5                   --
│    │    └─Conv2d: 3-9                  1,204,320
│    │    └─BatchNorm2d: 3-10            192
│    │    └─ReLU: 3-11                   --
│    │    └─MaxPool2d: 3-12              --
├─Sequential: 1-3                        --
│    └

In [32]:
x = torch.randn(1, 48).to(device)
print(x.shape)

x = model.adaptation(x)
print(x.shape)

x = model.unflat(x)
print(x.shape)

x = model.backbone(x)
print(x.shape)

print(x.shape[-1] * x.shape[-2] * x.shape[-3])

x = model.exit(x)
print(x.shape)

x = torch.randn(1, 48).to(device)
x = model(x)

torch.Size([1, 48])
torch.Size([1, 4096])
torch.Size([1, 1, 64, 64])
torch.Size([1, 96, 10, 10])
9600
torch.Size([1, 2])


In [ ]:
epochs = 5
batch_size = 300

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
lr=0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

import time
start_time = time.time()

seq = 0
for i in range(epochs):
    trn_cor = 0
    trn_cnt = 0
    tst_cor = 0
    tst_cnt = 0
        
    for b, (X_train, y_train) in enumerate(train_loader):
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        b+=1
            
        y_pred = model(X_train)

        # print(y_pred)
        # int(y_train)

        loss = criterion(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        predicted = torch.max(y_pred.data, 1)[1]
        batch_cor = (predicted == y_train).sum()
        trn_cor += batch_cor
        trn_cnt += len(predicted)

        cnf = torch.mean(torch.max(nn.functional.softmax(y_pred, dim=-1), 1)[0]).item()
            
        if (b-1)%10 == 0:
            print(f'Epoch: {i:2} Batch: {b:3} Loss: {loss.item():4.4f} Accuracy Train: {trn_cor.item()*100/trn_cnt:2.3f}%')
            
        seq += 1

print(f'\nDuration: {time.time() - start_time:.0f} seconds')